In [2]:
import pandas as pd
import numpy as np
import random
import pickle

from numpy import exp, sqrt, dot
from scipy.spatial.distance import cdist
from scipy.io import loadmat
from random import shuffle

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.model_selection import KFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor

from scipy.stats import kurtosis
from scipy.stats import moment
from scipy.stats import skew

import tensorflow as tf
from tensorflow.python.client import device_lib

import warnings
warnings.filterwarnings('ignore')

# Get the data into bootstrapped sequences tf.Dataset object

In [3]:
dataset = "MODIS"

In [18]:
def load_data():
    
    if dataset == "CORN" or dataset == "WHEAT":
        full_data = pickle.load(open(dataset + ".p", "rb"))
        columns = (["id"] +  ['label'] + ["reflectance_" + str(i) for i in range(92)])
        full_data.columns = columns
        
    else:
        mat_dict = loadmat('../MIR/' + dataset + '.mat')
        full_data = pd.DataFrame(mat_dict[dataset])

        # Rename columns to something more interpretable
        columns = (["id"] + ["reflectance_" + str(i) for i in range(7)]
                   + ["solar_" + str(i) for i in range(5)] + ['label'])
        full_data.columns = columns

        if dataset == "MISR2":
            full_data = full_data[full_data['id'].isin(list(full_data['id'].value_counts().index[full_data['id'].value_counts() == 100]))]

    return full_data

In [5]:
def moments_df(n_moments):
    
    full_data = load_data()

#     non_constant_cols = ['id', 'reflectance_0', 'reflectance_1', 'reflectance_2',
#            'reflectance_3', 'reflectance_4', 'reflectance_5', 'reflectance_6']
    
    non_constant_cols = (["id"] + ["reflectance_" + str(i) for i in range(12)])

    constant_columns = ['solar_0', 'solar_1', 'solar_2', 'solar_3']
    list_operation = ['np.mean', 'np.var', 'skew', 'kurtosis']

    full_data_subset = full_data[non_constant_cols]
    
    dic_df = {}

    for moments in range(1, n_moments+1):
        if moments < 5:
            dic_df[moments] = eval("np.stack(full_data_subset.groupby('id').apply(lambda group: " + 
                                   list_operation[moments-1] + "(group)).values)[:,1:]")
        else:
            dic_df[moments] = np.stack(full_data_subset.groupby('id').apply(lambda group: moment(group, moment=moments)))[:,1:]

    array_data = dic_df[1]
    
    if n_moments > 1:
        for key in range(2, n_moments+1):
            array_data = np.hstack([array_data, dic_df[key]])   

    df = pd.DataFrame(array_data)
    if dataset != "CORN" and dataset != "WHEAT":
        df = pd.concat([df, full_data.groupby('id').mean()[constant_columns].reset_index()], axis=1)
    df['id'] = full_data['id'].unique()
    df['label'] = full_data.groupby('id').mean()['label'].values
    df = df.set_index('id')
    
    col_label = ['label']
    labels = df[col_label]
    df = df[[col for col in df.columns if col not in col_label]]
    
    return df, labels

In [6]:
def make_sequences(train, test):
    list_sequence_train, list_sequence_test = [], []
    list_train_y, list_test_y = [], []
    features = [col for col in train.columns if col not in ['id', 'label']]
    
    for bag_id in list(train['id'].unique()):
        list_sequence_train.append(np.array(train[train['id']==bag_id][features]))
        list_train_y.append(np.array(train[train['id']==bag_id]['label'])[0])
        
    for bag_id in list(test['id'].unique()):
        list_sequence_test.append(np.array(test[test['id']==bag_id][features]))
        list_test_y.append(np.array(test[test['id']==bag_id]['label'])[0])
    
    return list_sequence_train, list_sequence_test, list_train_y, list_test_y

In [14]:
def inference_attention(random_seed, processor_steps, splits=5, use_moments=False, n_moments=1, concat_late=False):

    full_data = load_data()
    num_features = 12
    count = 1
    
    if use_moments:
        num_features = 92 + n_moments*92
        df, _ = moments_df(n_moments)
        col_solar = ['solar_0', 'solar_1', 'solar_2', 'solar_3', 'solar_4']
        df = df.iloc[np.repeat(np.arange(len(df)), 100)]
        df = df[[col for col in df.columns if col not in col_solar]]
        df = df.reset_index(drop=True)
        full_data = pd.concat([full_data, df], axis=1)
    
    kf = KFold(n_splits=splits, shuffle=True, random_state=random_seed)
    cols_exclude = ["id", "label"]
    features = [col for col in list(full_data.columns) if col not in cols_exclude]
    lowest_val_list, loss_test_list = [], []
    
    for train_index, test_index in kf.split(list(full_data['id'].unique())):
        #print('Compute for FOLD NUMBER ' + str(count))
        train_index = full_data['id'].unique()[train_index]
        test_index = full_data['id'].unique()[test_index]
        train = full_data[full_data['id'].apply(lambda value: value in train_index)]
        test = full_data[full_data['id'].apply(lambda value: value in test_index)]
        
        scaler = StandardScaler()
        scaler.fit(train[features])
        train[features], test[features] = scaler.transform(train[features]), scaler.transform(test[features])
    
        list_sequence_train, list_sequence_test, train_y, test_y = make_sequences(train, test)
        lowest_val_loss, loss_test = attention(list_sequence_train, list_sequence_test, train_y, test_y, 
                                               num_features, processor_steps, concat_late)
        
        lowest_val_list.append(lowest_val_loss)
        loss_test_list.append(loss_test)
        
        tf.reset_default_graph()
        count += 1
    
    return lowest_val_loss, loss_test_list

In [8]:
def compute_rmse(pred_array, truth_array):
    loss = np.sqrt(mean_squared_error(np.reshape(np.array(pred_array), (np.array(pred_array).shape[0],1)), 
                                               np.reshape(np.array(truth_array), (np.array(truth_array).shape[0],1))))
    return loss

In [9]:
# check the neighbours of the minimum to be sure it is not due to stochasticity
def get_min_index(array, n_max=3):
    indices = array.argsort()[:n_max] 
    
    avg_neighbourgs = []
    for val in indices:
        if val == len(array)-1:
            avg_neighbourgs.append((array[val]+array[val-1]+array[val-2])/3)
        else:
            avg_neighbourgs.append((array[val]+array[val-1]+array[val+1])/3)
    
    index = indices[np.argmin(avg_neighbourgs)]
    
    return index

# Build the network

In [52]:
# network hyperparameters

batch_size = 1
sequence_lenght = 100
dropout_p = 0.5

# reading block Neural Network size
size_h1 = 256

n_epochs = 500
num_test_runs = 1
learning_rate = 0.00002

In [53]:
def attention(list_sequence_train, list_sequence_val, train_y, val_y, num_features, processor_steps, concat_late=False):
    
    with tf.variable_scope('data'):
        
        prob = tf.placeholder_with_default(1.0, shape=())
        X_or = tf.placeholder(shape = [sequence_lenght, num_features], dtype = tf.float32, name = "input")
        if concat_late:
            X_or = tf.placeholder(shape = [sequence_lenght, 12], dtype = tf.float32, name = "input")
            X_fixed = tf.placeholder(shape = [1, num_features-12], dtype = tf.float32, name = "fixed_input")
            fixed_num_features = num_features-12
            num_features = 12
        y = tf.placeholder(shape = [1,1], dtype = tf.float32, name = "label_annotation")
        
    with tf.variable_scope('reading_block'):
    
        W_reading1 = tf.Variable(tf.random_normal(shape = (num_features, int(size_h1/2)), stddev = 0.1), name = "weights_hidden1", trainable = True)
        hidden_relu1 = tf.matmul(X_or, W_reading1)
        BN1 = tf.nn.relu(hidden_relu1, name = "ReLu_hidden_layer1")
        BN1 = tf.nn.dropout(BN1, prob, noise_shape=[1,int(size_h1/2)])
        
    with tf.variable_scope('initialize_lstm'):
        
        static_input = tf.zeros(shape=[1,1,1])
        unstack_input = tf.unstack(static_input, 1, 1)
        lstm_cell = tf.nn.rnn_cell.LSTMCell(size_h1, reuse=tf.AUTO_REUSE, state_is_tuple=True)
        initial_state = lstm_cell.zero_state(1, dtype=tf.float32)
        print(initial_state, initial_state[0], initial_state[-1])
        
    for index in range(processor_steps):
        with tf.variable_scope('lstm_cell_decoder', reuse = tf.AUTO_REUSE):
            
            output_decoder, state_decoder = tf.nn.static_rnn(lstm_cell, inputs=unstack_input,
                                                             initial_state=initial_state, dtype=tf.float32)
            
            print(output_decoder, state_decoder)
            print("for hidden/memory state: ", state_decoder[-1])

            W_decoder = tf.get_variable(initializer=tf.random_normal(shape = (size_h1, int(size_h1/2)), stddev = 0.1), name = "decoder_weights", trainable = True)

            relu_decoder = tf.nn.relu(tf.matmul(state_decoder[-1], W_decoder))
            relu_decoder = tf.nn.dropout(relu_decoder, prob, noise_shape=[1,int(size_h1/2)])

            inner_product = tf.matmul(BN1, tf.reshape(relu_decoder, shape=(int(size_h1/2), 1)))
            reshaped_inner_product = tf.reshape(inner_product, shape=(1, sequence_lenght))
#             W_attention = tf.get_variable(initializer=tf.random_normal(shape = (sequence_lenght, sequence_lenght), 
#                                                                        stddev = 0.1), name = "w_attention", trainable = True)
#             reshaped_inner_product = tf.nn.sigmoid(tf.matmul(reshaped_inner_product, W_attention))
            
            attention_score =  tf.nn.softmax(reshaped_inner_product)
            context_vector = tf.multiply(BN1, tf.reshape(attention_score, (sequence_lenght, 1)))
            r_vector = tf.reshape(tf.reduce_sum(context_vector, axis=0), (1,int(size_h1/2)))
            q_star = tf.concat([relu_decoder, r_vector], axis=1)

            initial_state = (initial_state[0], q_star)
            
    with tf.variable_scope('logits'):
        if concat_late:
            q_star_concat = tf.concat([q_star, X_fixed], axis=1)
            q_star_concat = tf.nn.dropout(q_star_concat, prob)
            W_concat = tf.Variable(tf.random_normal(shape = (size_h1+fixed_num_features, 1), stddev = 0.1), name = "weights_concat", trainable = True)
            logits = tf.nn.relu(tf.matmul(q_star_concat, W_concat))
        else:   
            W_logits = tf.Variable(tf.random_normal(shape = (size_h1, 1), stddev = 0.1), name = "weights_logits", trainable = True)
            logits = tf.nn.relu(tf.matmul(q_star, W_logits))
        
    with tf.variable_scope('loss'):
        
        loss =  tf.losses.mean_squared_error(predictions = logits, labels = y)
        
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(loss)
    
    print(tf.trainable_variables())
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    val_loss_list, test_loss_list = [], []

    for i in range(1, n_epochs+1):
    
        train_pred, train_truth = [], []
        val_test_pred, val_test_truth = [], []
        indices = list(range(len(list_sequence_train)))
        shuffle(indices)

        for index in indices:
            # batch is size 1 here: stochastic gradient descent
            if concat_late:
                X_batch_or = np.reshape(list_sequence_train[index][:,:12], (100, num_features))
                X_batch_fixed = np.reshape(list_sequence_train[index][0,12:], (1, fixed_num_features))
                Y_batch = np.reshape(train_y[index], (1,1))
                _, pred = sess.run([train_op, logits], feed_dict = {X_or: X_batch_or, X_fixed: X_batch_fixed,
                                                                    y: Y_batch, prob: dropout_p})
            else:
                X_batch = np.reshape(list_sequence_train[index], (100, num_features))
                Y_batch = np.reshape(train_y[index], (1,1))
                _, pred = sess.run([train_op, logits], feed_dict = {X_or: X_batch, y: Y_batch, prob: dropout_p})

            #_, pred = sess.run([train_op, logits], feed_dict = {X_or: X_batch, y: Y_batch, prob: dropout_p})
            train_pred.append(pred)
            train_truth.append(Y_batch)

        for index in range(len(list_sequence_val)):
            # batch is size 1 here: stochastic gradient descent
            if concat_late:
                X_batch_or = np.reshape(list_sequence_val[index][:,:12], (100, num_features))
                X_batch_fixed = np.reshape(list_sequence_val[index][0,12:], (1, fixed_num_features))
                Y_batch = np.reshape(val_y[index], (1,1))
            else:
                X_batch = np.reshape(list_sequence_val[index], (100, num_features))
                Y_batch = np.reshape(val_y[index], (1,1))
                
            #X_batch = np.reshape(list_sequence_val[index], (100, num_features))
            #Y_batch = np.reshape(val_y[index], (1,1))

            pred_avg = 0
            for _ in range(num_test_runs):
                if concat_late:
                    pred = sess.run([logits], feed_dict = {X_or: X_batch_or, X_fixed: X_batch_fixed,
                                                           y: Y_batch, prob: 1})
                else:
                    pred = sess.run([logits], feed_dict = {X_or: X_batch, y: Y_batch, prob: 1})
                pred_avg += pred[0]

            val_test_pred.append(pred_avg/num_test_runs)
            val_test_truth.append(Y_batch)
            
        if i == 1:
            # shuffle because the bags in the list are ranked (increasing order)
            range_arr = np.arange(len(val_test_truth))
            half_lenght = int(len(val_test_truth)/2)
            np.random.shuffle(range_arr)
            indices_val, indices_test = range_arr[:half_lenght], range_arr[half_lenght:]
        
        val_test_truth, val_test_pred = np.array(val_test_truth), np.array(val_test_pred)
        val_pred, val_truth = val_test_pred[indices_val], val_test_truth[indices_val]
        test_pred, test_truth = val_test_pred[indices_test], val_test_truth[indices_test]

        training_loss = compute_rmse(train_pred, train_truth)
        val_loss = compute_rmse(val_pred, val_truth)
        test_loss = compute_rmse(test_pred, test_truth)
    
        val_loss_list.append(val_loss)
        test_loss_list.append(test_loss)
        
        if (i == 1) or (i % 10 == 0):
            print("EPOCH " + str(i))
            print("TRAINING LOSS: " + str(training_loss))
            print("VAL LOSS: " + str(val_loss), "TEST LOSS: " + str(test_loss))

#     lowest_val_loss = min(val_loss_list)
#     loss_test = test_loss_list[val_loss_list.index(lowest_val_loss)]
    
    min_index = get_min_index(np.array(val_loss_list), 5)
    lowest_val_loss = val_loss_list[min_index]
    loss_test = test_loss_list[min_index]

    return lowest_val_loss, loss_test

In [59]:
# FOR DIFFERENT NUMBER OF MOMENTS AND THE SAME NUMBER OF PROCESSORS

iteration_number = 10
random_seed_list = list(range(iteration_number))
#number_of_moments = list(range(1,11))
number_of_moments = [4]
final_val_loss, final_test_loss = [], []

for moments in number_of_moments:
    print("COMPUTING FOR " + str(moments) + " MOMENTS")
    validation_loss_list, test_loss_list = [], []
    for index in range(iteration_number):
        print('COMPUTING FOR ITERATION NUMBER ' + str(index+1))
        lowest_val_loss, loss_test_list = inference_attention(random_seed_list[index], 2, 5, True, moments, False)
        validation_loss_list.append(np.mean(lowest_val_loss))
        test_loss_list.append(np.mean(loss_test_list))
        print(np.mean(lowest_val_loss), np.mean(loss_test_list))
        
    final_val_loss.append(np.mean(validation_loss_list))
    final_test_loss.append(np.mean(test_loss_list)) 
    print(np.mean(validation_loss_list), np.mean(test_loss_list))

In [58]:
# FOR NO MOMENTS WITH DIFFERENT PROCESSORS

iteration_number = 10
random_seed_list = list(range(iteration_number))
processor_steps_list = [2]
final_val_loss, final_test_loss = [], []

for num_processor_steps in processor_steps_list:
    print("COMPUTING FOR " + str(num_processor_steps) + " PROCESSING STEPS")
    validation_loss_list, test_loss_list = [], []
    for index in range(iteration_number):
        print('COMPUTING FOR ITERATION NUMBER ' + str(index+1))
        lowest_val_loss, loss_test_list = inference_attention(random_seed_list[index], num_processor_steps, 5, False, 1, False)
        validation_loss_list.append(np.mean(lowest_val_loss))
        test_loss_list.append(np.mean(loss_test_list))
        print(np.mean(lowest_val_loss), np.mean(loss_test_list))
        
    final_val_loss.append(np.mean(validation_loss_list))
    final_test_loss.append(np.mean(test_loss_list)) 
    print(np.mean(validation_loss_list), np.mean(test_loss_list))

In [56]:
tf.reset_default_graph()

In [35]:
print(tf.trainable_variables())

[]


In [29]:
# total_parameters = 0
# for variable in tf.trainable_variables():
#     # shape is an array of tf.Dimension
#     shape = variable.get_shape()
#     print(shape)
#     print(len(shape))
#     variable_parameters = 1
#     for dim in shape:
#         print(dim)
#         variable_parameters *= dim.value
#     print(variable_parameters)
#     total_parameters += variable_parameters
# print(total_parameters)

(12, 64)
2
12
64
768
(65, 256)
2
65
256
16640
(256,)
1
256
256
(128, 64)
2
128
64
8192
(128, 1)
2
128
1
128
25984


# Baselines

# 1. Aggregated-MIR (with moments)

In [274]:
def aggregated_method_moments(n_moments=1):
    
    random_seed_list = list(range(10))
    final_data, labels = moments_df(n_moments)
    
    final_loss_train, final_loss_test = [], []
    count = 1

    for index in range(10):    

        kf = KFold(n_splits=5, shuffle=True, random_state=random_seed_list[index])
        list_loss_train, list_loss_test = [], []
        for train_index, test_index in kf.split(range(labels.shape[0])):
            train_x, test_x = final_data.iloc[train_index,:], final_data.iloc[test_index,:]

            scaler = StandardScaler()
            scaler.fit(train_x)
            train_x, test_x = scaler.transform(train_x), scaler.transform(test_x)

            train_y, test_y = labels.iloc[train_index,:], labels.iloc[test_index,:]

            rbf_svr = MLPRegressor(hidden_layer_sizes=(512, ), learning_rate_init=0.001, 
                                   max_iter=300, alpha=1)
            
            rbf_svr.fit(train_x, train_y)
            train_pred = rbf_svr.predict(train_x)
            test_pred = rbf_svr.predict(test_x)

            training_loss = np.sqrt(mean_squared_error(np.reshape(np.array(train_pred),(len(train_pred),1)), np.reshape(np.array(train_y), (len(train_pred),1))))
            testing_loss = np.sqrt(mean_squared_error(np.reshape(np.array(test_pred),(len(test_pred),1)), np.reshape(np.array(test_y), (len(test_pred),1))))

            list_loss_train.append(training_loss)
            list_loss_test.append(testing_loss)

        final_loss_train.append(np.mean(list_loss_train))
        final_loss_test.append(np.mean(list_loss_test))
        print('Iteration number ' + str(count) + ' is done')
        count += 1

    print('The training loss is ' + str(np.mean(final_loss_train)))
    print('The test loss is ' + str(np.mean(final_loss_test)))
        

In [277]:
aggregated_method_moments(3)

Iteration number 1 is done
Iteration number 2 is done
Iteration number 3 is done
Iteration number 4 is done
Iteration number 5 is done
Iteration number 6 is done
Iteration number 7 is done
Iteration number 8 is done
Iteration number 9 is done
Iteration number 10 is done
The training loss is 11.284323043751655
The test loss is 35.60138319606699


# 2. Instance-MIR (with moments)

In [285]:
def instance_method(use_moments=False, n_moments=1):

    full_data = load_data()
    random_seed_list = list(range(10))
    
    if use_moments:
        df, _ = moments_df(n_moments)
        col_solar = ['solar_0', 'solar_1', 'solar_2', 'solar_3']
        df = df.iloc[np.repeat(np.arange(len(df)), 100)]
        df = df[[col for col in df.columns if col not in col_solar]]
        df = df.reset_index(drop=True)
        full_data = full_data.reset_index(drop=True)
        full_data = pd.concat([df, full_data], axis=1)

    final_loss_train, final_loss_test = [], []
    count = 1
    for index in range(10):

        kf = KFold(n_splits=5, shuffle=True, random_state=random_seed_list[index])
        cols_exclude = ["id", "label"]
        features = [col for col in list(full_data.columns) if col not in cols_exclude]
        list_loss_train, list_loss_test = [], []

        for train_index, test_index in kf.split(list(full_data['id'].unique())):
            train_index, test_index = np.array(full_data['id'].unique())[list(train_index)], np.array(full_data['id'].unique())[list(test_index)]
            train = full_data[full_data['id'].apply(lambda value: value in train_index)]
            test = full_data[full_data['id'].apply(lambda value: value in test_index)]

            scaler = StandardScaler()
            scaler.fit(train[features])
            train[features], test[features] = scaler.transform(train[features]), scaler.transform(test[features])

            train_x, train_y = train[features], train['label']
            test_x, test_y = test[features], test['label']

            rbf_svr = MLPRegressor(hidden_layer_sizes=(1024,), learning_rate_init=0.001, max_iter=300, alpha=3)
            rbf_svr.fit(train_x, train_y)
            train_pred = rbf_svr.predict(train_x)
            test_pred = rbf_svr.predict(test_x)

            df_train_pred = pd.DataFrame(np.concatenate([np.reshape(train_pred, (train_pred.shape[0],1)), 
                        np.reshape(train['id'].values, (train_pred.shape[0],1))], axis=1))

            df_test_pred = pd.DataFrame(np.concatenate([np.reshape(test_pred, (test_pred.shape[0],1)), 
                        np.reshape(test['id'].values, (test_pred.shape[0],1))], axis=1))

            true_train_y, true_test_y = train.groupby(['id']).mean()['label'].values, test.groupby(['id']).mean()['label'].values
            train_mean_pred = df_train_pred.groupby(1).mean().values
            test_mean_pred = df_test_pred.groupby(1).mean().values

            training_loss = np.sqrt(mean_squared_error(np.reshape(train_mean_pred,(train_mean_pred.shape[0],1)), 
                                                       np.reshape(true_train_y, (train_mean_pred.shape[0],1))))

            testing_loss = np.sqrt(mean_squared_error(np.reshape(test_mean_pred,(test_mean_pred.shape[0],1)), 
                                                      np.reshape(true_test_y, (test_mean_pred.shape[0],1))))

            list_loss_train.append(training_loss)
            list_loss_test.append(testing_loss)
            
        final_loss_train.append(np.mean(list_loss_train))
        final_loss_test.append(np.mean(list_loss_test))
        print('Iteration number ' + str(count) + ' is done')
        count += 1

    print('The training loss is ' + str(np.mean(final_loss_train)))
    print('The test loss is ' + str(np.mean(final_loss_test)))

In [286]:
instance_method(False, 1)

Iteration number 1 is done
Iteration number 2 is done
Iteration number 3 is done
Iteration number 4 is done
Iteration number 5 is done
Iteration number 6 is done
Iteration number 7 is done
Iteration number 8 is done
Iteration number 9 is done
Iteration number 10 is done
The training loss is 12.217802154190945
The test loss is 24.602675029887493
